# Table of Contents

* [Intialize](#Initalize)
* [Scrapper](#Scraper)
* [Clean Data](#Clean-Data)
* [Store as CSV](#Store-as-CSV)
* [Appendix](#Appendix)
  * [Load Data from Roster Dictionary](#Load-From-Roster)
  * [Load Data from CSV](#Load-From-CSV)

# Initalize

**Jump-to**: [Table of Content](#Table-of-Contents) | [Scrapper](#Scrapper) | [Clean Data](#Clean-Data) | [Store as CSV](#Store-as-CSV) | [Load Data from Roster Dictionary](#Load-From-Roster) | [Load Data from CSV](#Load-From-CSV)

In [ ]:
import pandas as pd
import numpy as np
import requests
import random
import time
from time import sleep
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait

from env import muser, mpass
print('Init')

# Scrapper

**Jump-to**: [Table of Content](#Table-of-Contents) | [Scrapper](#Scrapper) | [Clean Data](#Clean-Data) | [Store as CSV](#Store-as-CSV) | [Load Data from Roster Dictionary](#Load-From-Roster) | [Load Data from CSV](#Load-From-CSV)

In [ ]:
#code by pythonjar - grants browser permission 
chrome_options = webdriver.ChromeOptions()
prefs = {"profile.default_content_setting_values.notifications" : 2}
chrome_options.add_experimental_option("prefs",prefs)

#specify the path to chromedriver.exe 
PATH = 'C:/Program Files (x86)/chromedriver.exe'
#creates the driver object with the chrome driver and options as peramaters
driver = webdriver.Chrome(PATH, options=chrome_options)

#opens FB
driver.get("http://www.facebook.com")

# waits for FB to load and display the login boxes using CSS_selector types
username = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='email']")))
password = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='pass']")))

#clears the user name and password fields and fills them with FB credentials
username.clear()
username.send_keys(muser)
password.clear()
password.send_keys(mpass)

#waits 5 seconds or until the login button is displayed and then clicks it to login
WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))).click() 

#wait 30 seconds to get security code for 2way auth
time.sleep(30)

# wait a random amount of time for the page to load before trying to navigate to the group members page
# Note - if facebook limits this behavior, simulating clicks might be necessary
time.sleep(random.randrange(4, 11))

# create an empty list obj to store the list of dictionaries
rosters = []

# create a list of group member pages to pull rosters from replace the links inside the [] with the group you want to look at
groups = ['https://www.facebook.com/groups/2345147239035182/members',
          'https://www.facebook.com/groups/625463614476092/members',
          'https://www.facebook.com/groups/756220585201594/members'
         ]


# loop through the group urls
for group in groups:
    driver.get(group)
    time.sleep(random.randrange(5, 10))
    last_height = driver.execute_script("return document.body.scrollHeight")

# use a while true loop to scroll to the end of a group's member list before pulling the page_source
    while True:

        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page a random amount of time between 6 and 12 seconds
        time.sleep(random.randrange(6, 12))

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            # If heights are the same it will exit the function
            break
        last_height = new_height
    
    
    html = driver.page_source
    # create a soup object using html parser to use to prase the results
    soup = BeautifulSoup(html, "html.parser")
    # create a list of all div cards that contains the information for each field we want to dump into a DF
    # Note the class values are likely to change, check it yourself.
    divs = soup.findAll('div', class_='ow4ym5g4 auili1gw rq0escxv j83agx80 buofh1pr g5gj957u i1fnvgqd oygrvhab cxmmr5t8 hcukyx3x kvgmc6g5 nnctdnn4 hpfvmrgz qt6c0cv9 jb3vyjys l9j0dhe7 du4w35lb bp9cbjyn btwxx1t3 dflh9lhu scb9dxdr')
    
    # Create a for loop that breaks into each div (data card) and find each instance of each piece of info 
    for div in divs:
        names = div.findAll('a', class_='oajrlxb2 g5ia77u1 qu0x051f esr5mh6w e9989ue4 r7d6kgcz rq0escxv nhd2j8a9 nc684nl6 p7hjln8o kvgmc6g5 cxmmr5t8 oygrvhab hcukyx3x jb3vyjys rz4wbd8a qt6c0cv9 a8nywdso i1ao9s8h esuyzwwr f1sip0of lzcic4wl oo9gr5id gpro0wi8 lrazzd5p')
        joins = div.findAll('span', class_='d2edcug0 hpfvmrgz qv66sw1b c1et5uql lr9zc1uh e9vueds3 j5wam9gi lrazzd5p m9osqain')
        infos = div.findAll('span', class_='d2edcug0 hpfvmrgz qv66sw1b c1et5uql lr9zc1uh a8c37x1j keod5gw0 nxhoafnm aigsh9s9 d9wwppkn fe6kdd0r mau55g9w c8b282yb mdeji52x sq6gx45u a3bd9o3v knj5qynh pipptul6 hzawbc8m')    
        groups = div.findAll('span', class_='a8c37x1j ni8dbmo4 stjgntxs l9j0dhe7 ltmttdrg g0qnabr5 ojkyduve')
        
        # Create empty objects to store the pulled text in the for loops for each piece of data
        person = ''
        date = ''
        add_info = ''
        url = ''
        title = ''
                
        # loop through each div obj to pull the required information. This is required because the info is in list form
        # some fields are blank hence doing if statements in for loops
        for name in names:
            person = name.text
            url = 'https://www.facebook.com' + str(name.get('href'))
        for join in joins:
            if len(join.text) > 0:
                date = join.text
            else:
                date = 'none'
        for info in infos:
            if len(info.text) > 0:
                add_info = info.text
            else:
                add_info = 'none'
        for group in groups:
            title = group.text
        # create a dictionary that contains the information for each member from each div obj
        dicob = {'name':person,
                 'joined':date,
                 'add_info':add_info,
                 'user_group_profile':url,
                 'group':title}
        # append the dictionary to the list
        rosters.append(dicob)
    # extra sleep time before moving onto the next group
    time.sleep(8)

# close the driver
driver.close()

# Clean Data

**Jump-to**: [Table of Content](#Table-of-Contents) | [Scrapper](#Scrapper) | [Clean Data](#Clean-Data) | [Store as CSV](#Store-as-CSV) | [Load Data from Roster Dictionary](#Load-From-Roster) | [Load Data from CSV](#Load-From-CSV)

In [ ]:
# convert the roster list to a panda's data frame and drop any duplicate values
recruit = pd.DataFrame(rosters).drop_duplicates()
# drop any row where the name = 'Learn More', 'YOUR NAME', or is blank (name = '')
recruit = recruit[(recruit['name'] != 'Learn More') & (recruit['name'] != 'Anthony Rivera Straine') & (recruit['name'] != '')]
# reset the index and drop the original index as it's values do not matter
recruit.reset_index(drop=True, inplace=True)

# print the head of the DF
recruit.head()

Next, we have to split the link into links that go to the member's profile, the member's group profile, and the group link. After processing the links, we will add them to our original recurit DF

In [ ]:
# create a dataframe by splitting the link text into section from /
links = recruit['user_group_profile'].str.split('/', expand=True)
links.head()

In [ ]:
# In this instance, columns 1 and 7 are worthless and can be dropped
links.drop(columns=[1, 7], inplace=True)
# crate the member FB profile link. The 6th column is the FB user's unique ID.
# The ID can be used to access a their profile by appending it to "https://www.facebook.com/profile.php?id="
links['profile_link'] = "https://www.facebook.com/profile.php?id=" + links[6]
# Column 4 is the group's unique ID. A group can be accessed by appending the group id to "https://www.facebook.com/groups/"
links['group_link'] = "https://www.facebook.com/groups/" + links[4]
# The rest 
links = links[['profile_link', 'group_link']]
links.head()

In [ ]:
# then concatenate the recurit DF with the links DF to get the processed links into the DF
recruit = pd.concat([recruit, links], axis=1)

Now that we have all the useful links processed from the data, we can moveon to processing the Joined column. The joined column contains both an estimated join date and how they joined the group. For now, we will only process out an estaimated time_in_group column which will be a catagory variable of ***Less Than a Year*** or ***A Year or More***. We will also crate an acquired date column which will be the date the script was run for posterity.

---
**NOTE** The current code does not account for admins in groups or blank joined field. This will be remedied at a later date.

In [ ]:
# create a acquired_date column
recruit['aquired_date'] = pd.to_datetime('now').strftime("%Y-%m-%d")

# To create the time in group variable we will use np.where to find instances where joined does not (~) contains 'year'
# If year is present it will be A year or more and if not it will be a year or less. 
recruit['time_in_group'] = np.where(~recruit['joined'].str.contains('year'), 'Less Than a Year', 'A Year or More')

# Store as CSV

This finalizes the recurit DF (for now). From here we can store the results as a useable CSV

**Jump-to**: [Table of Content](#Table-of-Contents) | [Scrapper](#Scrapper) | [Clean Data](#Clean-Data) | [Store as CSV](#Store-as-CSV) | [Load Data from Roster Dictionary](#Load-From-Roster) | [Load Data from CSV](#Load-From-CSV)

In [ ]:
recuritment = pd.read_csv('Recruitment.csv')
print(recruit.shape[0])
print(recuritment.shape[0])
recruit = pd.concat([recuritment, recruit])

In [ ]:
recruit.to_csv('Recruitment.csv')

In [ ]:
recruit

----

# Appendix

### Load from Roster

Use the cell below if you've already acquired the roster List and want to turn it into a CSV

**Jump-to**: [Table of Content](#Table-of-Contents) | [Scrapper](#Scrapper) | [Clean Data](#Clean-Data) | [Store as CSV](#Store-as-CSV) | [Load Data from Roster Dictionary](#Load-From-Roster) | [Load Data from CSV](#Load-From-CSV)

In [ ]:
##########################################################
### cell for recaputring dataframe from stored rosters ###
##########################################################

# convert the roster list to a panda's data frame and drop any duplicate whole row values
recruit = pd.DataFrame(rosters).drop_duplicates()
# drop any row where the name = 'Learn More', 'your name', or is blank (name = '')
recruit = recruit[(recruit['name'] != 'Learn More') & (recruit['name'] != 'Anthony Rivera Straine') & (recruit['name'] != '')]
# reset the index and drop the original index as it's values do not matter
recruit.reset_index(drop=True, inplace=True)


recruit.to_csv("Recruitment.csv")

### Load From CSV

Use the cell below if you've already acquired the CSV and want to make changes to it

**Jump-to**: [Table of Content](#Table-of-Contents) | [Scrapper](#Scrapper) | [Clean Data](#Clean-Data) | [Store as CSV](#Store-as-CSV) | [Load Data from Roster Dictionary](#Load-From-Roster) | [Load Data from CSV](#Load-From-CSV)

In [ ]:
######################################################
### cell for recaputring dataframe from stored csv ###
######################################################

recruit = pd.read_csv('Recruitment.csv')

----